# FMCG Forecasting: Feature Store & Inference

This notebook performs the following key steps in the machine learning pipeline:

1. **Prepare and register features**  
   - Use engineered daily features for SKU **MI-006**
   - Ensure compatibility with Databricks Feature Store
   - Create or overwrite a feature table with primary keys

2. **Train or load a production-ready model**  
   - Use Random Forest or previously trained model
   - Optionally register the model in the Databricks Model Registry

3. **Run inference on historical data**  
   - Join new batch data with feature table
   - Predict daily `units_sold` using the registered model

4. **Log predictions and optionally update Feature Store**  
   - Save batch predictions in Parquet format
   - Append predictions to the feature table for future retraining

> This notebook simulates how machine learning teams in retail or FMCG use **Feature Store + Model Registry** for scalable, reproducible, and traceable prediction workflows.


## Imports


In [0]:
# PySpark + MLflow + Feature Store
from pyspark.sql.functions import col
from databricks.feature_store import FeatureStoreClient
import mlflow


## Create features | save to Feature Store

In [0]:
# Initialize Feature Store Client
fs = FeatureStoreClient()


In [0]:
from utils.forecasting_utils import engineer_features_daily
path_daily = "dbfs:/FileStore/fmcg/parquet/FMCG_2022_2024.parquet"
# Daily
df_daily = spark.read.parquet(path_daily).filter(col("sku") == "MI-006")

df_daily_fe = engineer_features_daily(df_daily)
df_daily_fe = df_daily_fe.dropna()

In [0]:
# Define feature table name
feature_table_name = f"fmcg_features_daily"

# Drop if exists (for tests)
try:    table = fs.get_table(name=feature_table_name)
except Exception:
    table = None
if table:
    fs.drop_table(name=feature_table_name)

    fs.drop_table(name=feature_table_name)


# Create feature table in Feature Store
fs.create_table(
    name=feature_table_name,
    primary_keys=["sku", "date","channel", "region","pack_type"],
    df=df_daily_fe,
    schema=df_daily_fe.schema,
    description="Daily engineered features for FMCG forecasting"
)
